In [1]:
from fmm import Network,NetworkGraph,FastMapMatch,FastMapMatchConfig,UBODT,GPSConfig,ResultConfig
import pandas as pd
from shapely.wkt import loads
from shapely.geometry import mapping
import json
from ipyleaflet import Map, GeoJSON, WidgetControl
import ipywidgets as widgets

In [2]:
#load network data and graph
network = Network("../osmnx_example/beijing/edges.shp","fid", "u", "v")
print("Nodes {} edges {}".format(network.get_node_count(),network.get_edge_count()))
graph = NetworkGraph(network)

Nodes 157738 edges 393216
[2025-02-22 20:06:03.469] [info] [network.cpp:72] Read network from file ../osmnx_example/beijing/edges.shp
[2025-02-22 20:06:07.219] [info] [network.cpp:172] Number of edges 393216 nodes 157738
[2025-02-22 20:06:07.219] [info] [network.cpp:173] Field index: id 18 source 0 target 1
[2025-02-22 20:06:07.514] [info] [network.cpp:176] Read network done.
[2025-02-22 20:06:07.515] [info] [network_graph.cpp:17] Construct graph from network edges start
[2025-02-22 20:06:07.566] [info] [network_graph.cpp:30] Graph nodes 157738 edges 393216
[2025-02-22 20:06:07.566] [info] [network_graph.cpp:31] Construct graph from network edges end


### Precompute an UBODT file

**This step can be skipped if you already created one UBODT file.**

In [ ]:
from fmm import UBODTGenAlgorithm
ubodt_gen = UBODTGenAlgorithm(network,graph)
status = ubodt_gen.generate_ubodt("../osmnx_example/beijing/ubodt.txt", 4, binary=False, use_omp=True)
print(status)

[2025-02-21 12:06:36.625] [info] [ubodt_gen_algorithm.cpp:76] Start to generate UBODT with delta 4
[2025-02-21 12:06:36.625] [info] [ubodt_gen_algorithm.cpp:77] Output format csv
Status: success
Time takes 0.82 seconds

[2025-02-21 12:06:36.708] [info] [ubodt_gen_algorithm.cpp:105] Progress 64 / 649
[2025-02-21 12:06:36.787] [info] [ubodt_gen_algorithm.cpp:105] Progress 128 / 649
[2025-02-21 12:06:36.866] [info] [ubodt_gen_algorithm.cpp:105] Progress 192 / 649
[2025-02-21 12:06:36.951] [info] [ubodt_gen_algorithm.cpp:105] Progress 256 / 649
[2025-02-21 12:06:37.046] [info] [ubodt_gen_algorithm.cpp:105] Progress 320 / 649
[2025-02-21 12:06:37.125] [info] [ubodt_gen_algorithm.cpp:105] Progress 384 / 649
[2025-02-21 12:06:37.199] [info] [ubodt_gen_algorithm.cpp:105] Progress 448 / 649
[2025-02-21 12:06:37.278] [info] [ubodt_gen_algorithm.cpp:105] Progress 512 / 649
[2025-02-21 12:06:37.370] [info] [ubodt_gen_algorithm.cpp:105] Progress 576 / 649
[2025-02-21 12:06:37.441] [info] [ubodt_gen

### FMM model creation and map matching

match single trajectory

In [15]:
df = pd.read_csv("../osmnx_example/beijing/trajectories.csv", sep=";")

#trajectory 1
wkt = df.loc[df["id"] == 1, "geom"].values[0]

ubodt = UBODT.read_ubodt_csv("../osmnx_example/beijing/ubodt.txt")
model = FastMapMatch(network,graph,ubodt)

# configuration parameters
k = 4
radius = 0.4
gps_error = 0.5
fmm_config = FastMapMatchConfig(k,radius,gps_error)

result = model.match_wkt(wkt,fmm_config)
print("Matched path: ", list(result.cpath))
print("Matched edge for each point: ", list(result.opath))
print("Matched edge index ",list(result.indices))
print("Matched geometry: ",result.mgeom.export_wkt())
print("Matched point ", result.pgeom.export_wkt())

[2025-02-22 20:57:18.019] [info] [ubodt.cpp:208] Reading UBODT file (CSV format) from ../osmnx_example/beijing/ubodt.txt
[2025-02-22 20:57:19.068] [info] [ubodt.cpp:236] Read rows 1000000
[2025-02-22 20:57:19.859] [info] [ubodt.cpp:236] Read rows 2000000
[2025-02-22 20:57:20.640] [info] [ubodt.cpp:236] Read rows 3000000
[2025-02-22 20:57:21.401] [info] [ubodt.cpp:236] Read rows 4000000
[2025-02-22 20:57:22.148] [info] [ubodt.cpp:236] Read rows 5000000
[2025-02-22 20:57:23.074] [info] [ubodt.cpp:236] Read rows 6000000
[2025-02-22 20:57:24.080] [info] [ubodt.cpp:236] Read rows 7000000
[2025-02-22 20:57:24.888] [info] [ubodt.cpp:236] Read rows 8000000
[2025-02-22 20:57:25.832] [info] [ubodt.cpp:236] Read rows 9000000
[2025-02-22 20:57:26.633] [info] [ubodt.cpp:236] Read rows 10000000
[2025-02-22 20:57:27.387] [info] [ubodt.cpp:236] Read rows 11000000
[2025-02-22 20:57:28.295] [info] [ubodt.cpp:236] Read rows 12000000
[2025-02-22 20:57:29.185] [info] [ubodt.cpp:236] Read rows 13000000
[202

In [ ]:
import folium
from shapely.wkt import loads

original_geom = loads(wkt)
matched_geom = loads(result.mgeom.export_wkt())

m = folium.Map(location=[original_geom.coords[0][1], original_geom.coords[0][0]], zoom_start=16)

folium.PolyLine([(lat, lon) for lon, lat in original_geom.coords], color="red", weight=3, opacity=0.8, tooltip="Original GPS Trajectory").add_to(m)
folium.PolyLine([(lat, lon) for lon, lat in matched_geom.coords], color="blue", weight=3, opacity=0.8, tooltip="Matched Path").add_to(m)

for lon, lat in original_geom.coords:
    folium.CircleMarker(location=[lat, lon], radius=3, color="red", fill=True, fill_color="red").add_to(m)
for lon, lat in matched_geom.coords:
    folium.CircleMarker(location=[lat, lon], radius=3, color="blue", fill=True, fill_color="blue").add_to(m)

# Display map 
m

### Map Matching after obfuscation